In [1]:
import requests
import json
import pandas as pd

df_club_Season = pd.read_csv("season_club.csv")
df_result = pd.DataFrame(columns=['club_id', 'season_id', 'player_id'])
df_result['club_id'] = df_club_Season['club_id'].str.replace("c-", "")

players_data = []
seasons = [ 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

# Get unique club IDs
unique_club_ids = df_result['club_id'].unique()
i=1
for club in unique_club_ids:
    for season in seasons:
        print(i,club,season)
        i+=1
        url = f"https://transfermarkt-api.vercel.app/clubs/{club}/players?season_id={season}"
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36'}
        response = requests.get(url, headers=headers)
        
        try:
            json_data = response.json()
        except json.decoder.JSONDecodeError:
            print(f"Error: Failed to decode JSON response for club_id={club}, season_id={season}")
            continue
        
        for player in json_data.get("players", []):
            players_data.append(player.get('id'))

print(players_data)


FileNotFoundError: [Errno 2] No such file or directory: 'season_club.csv'

In [ ]:
playerIdList=players_data
# playerIdList = [108390, 40204, 5658, 59377, 50219, 17965, 101118, 52570, 71271, 33027, 16911, 128899, 29692, 47659, 45494, 39471, 29712, 13572, 47659, 15220]
data = []

for player_id in playerIdList:
    url = f"https://transfermarkt-api.vercel.app/players/{player_id}/transfers"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36'}
    response = requests.get(url, headers=headers)
    str_form = response.content.decode('utf-8')
    json_data = json.loads(str_form)

    for transfer in json_data["history"]:
        transfer_data = {
            "id": json_data["id"],
            "transfer_season": transfer["transferDate"],
            "old_club_id": transfer["oldClubID"],
            "new_club_id": transfer["newClubID"],
            "market_value": transfer["marketValue"],
            "fee": transfer["fee"]
        }
        data.append(transfer_data)

df = pd.DataFrame(data)
df.to_csv('transfers.csv', index=False)

In [ ]:
import pandas as pd


df = pd.read_csv('transfers_date.csv')


df['transfer_season'] = pd.to_datetime(df['transfer_season']).dt.year


df['id'] = 'p-' + df['id'].astype(str)


df['old_club_id'] = 'c-' + df['old_club_id'].astype(str)
df['new_club_id'] = 'c-' + df['new_club_id'].astype(str)


df['market_value'] = df['market_value'].replace(['free transfer', 'loan', '-', '?'], None)
df['market_value'] = df['market_value'].str.replace('€', '')
df['market_value'] = df['market_value'].str.replace('k', 'e3')
df['market_value'] = df['market_value'].str.replace('m', 'e6')
df['market_value'] = pd.to_numeric(df['market_value'], errors='coerce')


df['fee'] = df['fee'].replace(['free transfer', 'loan', '-', '?'], None)
df['fee'] = df['fee'].str.replace('€', '')
df['fee'] = df['fee'].str.replace('k', 'e3')
df['fee'] = df['fee'].str.replace('m', 'e6')
df['fee'] = pd.to_numeric(df['fee'], errors='coerce')


df.to_csv('transfers_modified.csv', index=False)